# Graphical Models: Part 1

###### COMP4670/8600 - Statistical Machine Learning - Tutorial

### Assumed knowledge
- Directed graphical models (Bayesian networks)
- Conditional independence, joint distribution factorisation
- D-separation and proving conditional (in)dependence based on D-separation.

### After this lab, you should be comfortable with
- Basic operations and definitions of graphical models

Setting up the environment

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Image

This tutorial mainly includes three parts: Bayesian Networks (BN), Markov Random Field (MRF) and Sum Product Algorithm (Factor Graph). Before diving into the graphical models, we will first review some basic probability concepts. 

## Probability Revision

Ensure that you are familiar with the following terms:
- Joint probability distribution
- Marginal distribution
- Conditional distribution

Consider the following table defining the joint probability distribution of two variables $A \in \{1,2,3,4,5\}$ and $B \in \{p, q, r\}$.

|  | A=$1$ | A=$2$ | A = $3$ | A = $4$ | A = $5$ |
|--|:--:|:--:|:--:|:--:|:--:|
|**B**=$p$|0.01|0.01|0.12|0.01|0.14|
|**B**=$q$|0.03|0.15|0.01|0.01|0.01|
|**B**=$r$|0.13|0.11|0.07|0.18|0.01|

Below is the table in Python.

In [ ]:
P_AB = pd.DataFrame([[0.01, 0.01, 0.12, 0.01, 0.14],
                     [0.03, 0.15, 0.01, 0.01, 0.01],
                     [0.13, 0.11, 0.07, 0.18, 0.01]],
                    index=["p", "q", "r"],
                    columns=[1,2,3,4,5])

print("Table for p(A, B)\n")
print(P_AB)

# Check the table is non-negative
assert (P_AB >= 0).all().all()
# Check if the table sums to 1
assert np.allclose(P_AB.sum().sum(), 1)

**Exercise 1.** Compute the following probabilities:
- $p(B)$
- $p(B \mid A = 2)$
- $p(B = p \mid A = 2)$

In [ ]:
# replace this with your solution, add and remove code and markdown cells as appropriate

### Empirical verification of Bayes rule

Given the joint distribution $p(A,B)$ there are two ways to compute the posterior $p(A \mid B)$.

The first way is by using the definition of conditional probability: $p(A \mid B) = \frac{p(A, B)}{p(B)}$.

The second way is by using the Bayes rule: $p(A \mid B) = \frac{p(B \mid A)p(A)}{\sum_A p(A,B)}$.

**Exercise 2.** Compute the tables for $p(A \mid B)$ for all values of $A$ and $B$ using both ways. Ensure that the results are the same.

In [ ]:
# replace this with your solution, add and remove code and markdown cells as appropriate

### Dependent random variables

Consider the following 5 random variables.
- **A**ches with states (False, True)
- **B**ronchitis with states (none, mild, severe)
- **C**ough with states (False, True)
- **D**isease with states (healthy, carrier, sick, recovering)
- **E**mergency with states (False, True)

**Exercise 3.** How much memory is needed to store the joint probability distribution if:

(a) All variables are dependent?

(b) All variables are mutually independent?

What does this tell us about the benefit of establishing independencies among a set of random variables?

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

## Bayesian Network

Bayesian Network is directed graphical model expressing causal relationship between variables.

Consider the following graphical model with variables described in Exercise 3.

In [ ]:
Image(url="https://machlearn.gitlab.io/sml2021/tutorials/graphical_model.png")

**Exercise 4.** Answer the following questions:
1. Write down the joint factorisation  for the above graph. 
2. How much memory is needed to store the joint probability distribution?

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

### Conditional Independence  and D-seperation

Conditional independence is an important notion in graphical models. Consider three sets of nodes $X$, $Y$ and $Z$. $X$ is said to be conditionally independent of $Y$, given $Z$ (written as $X \perp\!\!\!\perp Y | Z$)  if
$$
p(X, Y \mid Z) = p(X \mid Z) p(Y \mid Z).
$$


So, given a joint distribution characterised by a directed graphical model, how do we prove/disprove that two (sets of) variables are independent conditioned on another (set of) variables? The lecture introduced us to two ways of doing so:
- From the joint distribution, marginalise all irrelevant variables. We should now have $p(X, Y, Z)$. Then show that, based on the structure of the graph, we can factorise $p(X, Y \mid Z)$ into $p(X \mid Z) p(Y \mid Z)$. An equivalent way is to show $p(X \mid Y, Z) = p(X \mid Z)$.
- (D-separation) Consider all *undirected* paths from any node in $X$ to any node in $Y$. $X$ is conditionally independent of $Y$ given $Z$ if and only if every such path is *blocked*, that is, the path includes a node such that
 - The node is in set $Z$ and it is HT or TT, or
 - The node is HH, but neither the node nor any of its descendants is in set $Z$.

**Example proof.**
Below is an example to proof using D-separation that in the above graph $C \perp\!\!\!\perp E | D$.

*(Using the first way)*
We will show that $p(E \mid C, D) = p(E \mid D)$. Exploiting the structure of the graph, we have:
$$
\begin{align*}
p(E \mid C, D) = \frac{p(E, C, D)}{p(C, D)}.
\end{align*}
$$
The numerator is
$$
\begin{align*}
p(E, C, D) & = \sum_{b} p(E \mid D) p(D \mid C, b) p(b) p(C) \\
& = p(C) p(E \mid D) \sum_{b} p(D \mid C, b) p(b) \\
& = p(C) p(E \mid D) \sum_{b} p(D, b \mid C) \\
& = p(C) p(E \mid D) p(D \mid C).
\end{align*}
$$
The denominator is
$$
\begin{align*}
p(C, D) & = \sum_{b} p(D \mid C, b) p(b) p(C) \\
& = p(C) \sum_{b} p(D \mid C, b) p(b) \\
& = p(C) \sum_{b} p(D, b \mid C) \\
& = p(C) p(D \mid C).
\end{align*}
$$
Therefore, $p(E \mid C, D) = \frac{p(C) p(E \mid D) p(D \mid C)}{p(C) p(D \mid C)} = p(E \mid D)$, which by definition implies that $E$ is independent of $C$ conditioned on $D$.

*(Using D-separation)*
There is only one (undirected) path from $C$ to $E$, which is $C \rightarrow D \rightarrow E$. This path contains node $D$, which is observed and is a HT node. Therefore, this path must be blocked. Since all paths between $C$ and $D$ are blocked, $C \perp\!\!\!\perp E | D$.

**Exercise 5.** Identify and prove whether the conditional independences holds for the following cases: 

(a) A and D, when B is observed.

(b) B and C, when none of the variables are observed.

(c) B and C, when E is observed.

(d) A and C, when none of the variables are observed.

(e) A and C, when B is observed.

(f) A and E, when D is observed.

### Calculating distributions for BN

**Exercise 6.** Consider the following tables.

|p(B) | B=n | B=m | B=s |
|:-----:|:--:|:--:|:--:|
|marginal| 0.97 | 0.01 | 0.02 |

|p(C) | C=False | C=True |
|:-----:|:--:|:--:|
|marginal| 0.7 | 0.3 |

| p(A\|B) | B=n | B=m | B=s |
|:-----:|:--:|:--:|:--:|
|**A**=False |0.9|0.8|0.3|
|**A**=True |0.1|0.2|0.7|

| p(D\|B,C) | B=n, C=F | B=m, C=F | B=s, C=F | B=n, C=T | B=m, C=T | B=s, C=T |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|
|**D**=healthy   |0.9 |0.8 |0.1 |  0.3 |0.4 |0.01|
|**D**=carrier   |0.08|0.17|0.01|  0.05|0.05|0.01|
|**D**=sick      |0.01|0.01|0.87|  0.05|0.15|0.97|
|**D**=recovering|0.01|0.02|0.02|  0.6 |0.4 |0.01|


| p(E\|D) | D=h | D=c | D=s | D=r |
|:-----:|:--:|:--:|:--:|:--:|
|**E**=False | 0.99 | 0.99| 0.4| 0.9|
|**E**=True | 0.01| 0.01| 0.6| 0.1|

Compute the following:
- $p(A,B,C,D,E)$
- $p(E)$
- $p(E \mid B=s)$
- $p(E \mid B=s, C=T)$

Note that there are two ways of arriving at the distributions:
1. By computing p(A,B,C,D,E) and marginalising and conditioning appropriately
2. By only computing the required distributions directly using the graphical model structure.

### <span style="color:blue">Answer</span>
<i>--- replace this with your solution, add and remove code and markdown cells as appropriate ---</i>

### Textbook Questions

These exercises also include ones about Markov random fields, which we will cover next week.

- Q8.20: Induction on graph structure (recall from MATH1005/6005) (Difficulty $\star$)
- Q8.21: Note typo: it should be $f_s(x_s)$ (Difficulty $\star\star$)
- Q8.27: Construct example showing greedy method not working (Difficulty $\star\star$)
- Q8.29: Induction on tree structure (recall from MATH1005/6005) (Difficulty $\star\star$)
- Extra: Derive eq 8.74 to 8.85 w.r.t Fig 8.51

- Q10.2: Solving simulataneous equations (Difficulty $\star$)
- Q10.3: Use lagrangian to enforce normalisation of q (Difficulty $\star\star$)
- Q10.6: Hint, how to introduce log term for both p and q? (Difficulty $\star\star$)

- Q2.44: Manipulation with a more complex conjugate to derive the posterior (Difficulty $\star\star$)

- Q10.7: Rewritting to the form of the respective distributions. Mostly algebra. (Difficulty $\star\star$)
- Q10.8: What will $b_n$ be approximated as? (Difficulty $\star$)
- Q10.9: Essentially, deriving 10.31 and 10.32 (Difficulty $\star\star$)

